In [1]:
%pip install customized-koNLPy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#반복문의 진행 상황을 시각적으로 표시 가능
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json

#from konlpy.tag import Okt, Komoran, Kkma
from ckonlpy.tag import Twitter
from gensim.models import Doc2Vec # gensim의 doc2vec을 사용해 수치적 표현으로 변환하는 것을 목푲
from gensim.models.doc2vec import TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec #모델 트레이닝 과정 모니터링 
from sklearn.metrics.pairwise import cosine_similarity #유사도 측정 
import numpy as np
from tqdm import tqdm
import os  # 모델이나 데이터 구조 저장 및 로드 
import pickle # 모델이나 데이터 구조 저장 및 로드 
import pandas as pd 

In [4]:
# 데이터 경로 설정
data_path = './items.json'

In [5]:
# 데이터 로드 후 데이터프레임 확인
df = pd.read_json(data_path)

In [6]:
# '_source' 키를 통해 데이터프레임 생성
#normalize:  정규화
df = pd.json_normalize(df['_source']) #엘라스틱의 검색 인덱스가 _soucre 아래에 저장된 데이터를 포함 

In [7]:
tokenizer = Twitter()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [68]:
#연속된 문자열 제거. 
#ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이런 문자로 인해 kkma()런타임 오류남
def remove_consecutive_words(s): # 연속해서 3번이상 반복되는 문자열 제거 반혼 'aaabbb' -> aabbc'
    result = [] 
    count = 1  

    if s:
        result.append(s[0])

    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            count += 1
        else:
            count = 1

        if count < 3:
            result.append(s[i])

    return ''.join(result)


#붙여쓴 문장을 강제로 분리하기. 길어지면 kkma()런타임 오류남
def insert_spaces(input_string, max_word_length=20):
    words = input_string.split()

    for i, word in enumerate(words):
        if len(word) >= max_word_length:
            split_words = [word[j:j+max_word_length] for j in range(0, len(word), max_word_length)]
            words[i] = '. '.join(split_words)

    result_string = ' '.join(words)
    return result_string



#토큰나이징 
# 토큰화 과정의 일부로서, 특정 문서나 문장을 토큰화하기 전에 필요한 사전 처리를 목적
def tokenizing(sentence):
    sentence = insert_spaces(sentence.lower())
    sentence = remove_consecutive_words(sentence)
    tokens = tokenizer.nouns(sentence)
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x for x in tokens if not x.isdigit()]
    return tokens

# TaggedDocument 형태로 변환하는 함수
#dataFrame의 각 행을 순회하며, item_idx 및 contents 열의 데이터를 사용하여 처리를 수행하도록 설계
#특정 칼럼의 텍스트 데이터를 형태소 분석하여 토큰화 한 다음, 이를 TaggedDocument 객체로 만드는 과정을 수행합니다. 
#여기서 TaggedDocument는 Doc2Vec 모델 학습에 사용되는 데이터 구조로, 각 문서(또는 문장)의 단어 리스트와 해당 문서에 할당된 태그(식별자)를 포함합
def tag_sentences_from_df(df): # 데이터프레임을 인자로 받는다 
    tagged_data = []
    for _, row in df.iterrows():
        item_idx = row['item_idx']  # 각 문서에 대한 고유 ID를 변수에 저장
        processed_text = row['combined']  # 전처리된 문서 내용을 변수에 저장
        
        if not processed_text:
            continue
        tokens = tokenizing(processed_text)  # 전처리된 텍스트를 토큰화
        tagged_data.append(TaggedDocument(words=tokens, tags=[f"{item_idx}"]))
    return tagged_data

#tagged_data : 각 문서의 토큰화된 단어들(words)과 각 문서를 구볋할 수 있는 식별자 (tags)를 포함하는 taggedocument 객체들의 리스트 

In [69]:
with open("user_dictionary_stringify.txt", 'r', encoding='utf-8') as file:
    user_dict = file.read().split(",")

user_dict = sorted(user_dict, key=len, reverse=True)

for word in user_dict:
    tokenizer.add_dictionary(word, 'Noun')

In [67]:
# DataFrame에 'combined' 열 추가
df['combined'] = df.apply(lambda row: row["subject"] + " " + row["contents"], axis=1) 

# 텍스트 데이터 전처리
# 'combined' 열에 대해 연속된 문자열 제거 및 필터링 수행
df['combined'] = df['combined'].apply(remove_consecutive_words)
# 길이가 300자를 초과한다면 그 항목을 300자까지로 자름. 60자 이하라면 none으로 설정
df['combined'] = df['combined'].apply(lambda x: x[:300] if len(x) > 60 else None) 

# 필터링된 데이터만 유지
df = df.dropna(subset=['combined'])



# Doc2Vec 모델 학습 데이터 준비
if not os.path.exists("doc2vec_corpus2.pkl"):
    tagged_data = tag_sentences_from_df(df[['item_idx', 'combined']]) # 데이터프레임 유지
    with open("doc2vec_corpus2.pkl", "wb") as file:  # 파일 이름 오타 수정
        pickle.dump(tagged_data, file)
else:
    with open("doc2vec_corpus2.pkl", "rb") as file:
        tagged_data = pickle.load(file)


In [88]:
print(df.columns)

Index(['item_idx', 'subject', 'contents', 'author_nick', 'created_at',
       'replies', 'combined', 'vector'],
      dtype='object')


In [87]:
# DataFrame의 처음 5행을 출력합니다.
print(df.head())


   item_idx                 subject  \
0      4561        토토로와 그녀의 청담동 나들이   
1      5684   ILE와 PEAI 입학시험 체험기...   
2       191                아들아 보거라.   
3      7408  이안어학원 영역별 수강시스템 괜찮을까요?   
4      8194    공부밖에 몰랐던 전교1등 이야기 #6   

                                            contents author_nick  \
0  청담安 이라고 들어보셨나요 ?이미 많이 알려진 강남의 명소로 개그맨(?) 윤정수씨가...         백지수   
1  제가 대치동에 오기 전에는 들어가기 힘든 순서가 렉스킴&gt;피아이&gt;아이엘이로...         박혀하   
2  한미은행장이 아들에게 보낸 편지입니다.공감을 주는 내용이 많아서 인터넷에 회자되었던...         양유성   
3  예비중1 이안어학원에 대한 들리는 평판이 좋기도 했고, 또 마침 아이시간표 조절할일...         류태근   
4  아이템풀과 구몬수학  누군가 인생은 괴로움의 연속이라고 했던가. 원이형도 같은 반 ...         김재재   

            created_at                                            replies  \
0  2009-08-08 18:44:45  [1. 저 데이트에 끼고 싶습니다.2. 혹시 설마 오렌지족이셨습니까?3. 믿고 의지...   
1  2009-11-13 06:54:20  [글쎄...저도 ILE가봤는데 저희아이는 5학년이라 그런지 모르겠지만 모든 게 다 ...   
2  2008-08-26 09:56:50  [좋은글을 많이 만나봤지만 우리들 모두에게 마음으로 필요한 말인것같아 넘 감사드립니...   
3  2010-02-09 21:40:03  [저는 영어종합반 보다는 영역별 개설 강좌를

In [86]:
print(df.tail())

        item_idx                                            subject  \
122440    545008  중등아이가 친구랑놀다가 4번째손가락을 골절되었는데요이동네 정형외과에 가서 손가락 기...   
122441    544947  초5 생크 중학과정하는데..개념원리 일품 최상위 3권해요. 그런데..이게 숙제하느라...   
122442    544919  중2사춘기에 접어든거같은데요모두알아서 한다고 해서 학원숙제 조금하고 간게2주고 그뒤...   
122444    545102  올해 입학한 대학생 딸아이가 있어요. 그동안은 성격이 지랄맞긴해도 관계가 나쁘진 않...   
122445    545095  친구가 저더러 자식하고 감정분리가 안되는게 문제라고 합니다. 친구는 외국에서 아이들...   

                                                 contents author_nick  \
122440  중등아이가 친구랑놀다가 4번째손가락을 골절되었는데요이동네 정형외과에 가서 손가락 기...         권웅원   
122441  초5 생크 중학과정하는데..개념원리 일품 최상위 3권해요. 그런데..이게 숙제하느라...         김미하   
122442  중2사춘기에 접어든거같은데요모두알아서 한다고 해서 학원숙제 조금하고 간게2주고 그뒤...         오상타   
122444  올해 입학한 대학생 딸아이가 있어요. 그동안은 성격이 지랄맞긴해도 관계가 나쁘진 않...         이재면   
122445  친구가 저더러 자식하고 감정분리가 안되는게 문제라고 합니다. 친구는 외국에서 아이들...         백산재   

                 created_at  \
122440  2023-08-12 13:19:18   
122441  2023-08-12 07:27:21   
122442  2023-08-11 22:00:15   
122444  20

In [73]:
# len 함수를 사용하여 행의 수 확인
print(len(df))

# shape 속성의 첫 번째 요소를 사용하여 행의 수 확인
print(df.shape[0])


82119
82119


In [82]:
missing_values_count = df.isnull().sum()

# 각 열별로 누락된 값의 개수 출력
print(missing_values_count)

# 전체 데이터 프레임에서 누락된 값의 총합 출력
total_missing_values = missing_values_count.sum()
print(f"데이터 프레임 전체에서 누락된 값의 총 개수: {total_missing_values}")

item_idx       0
subject        0
contents       0
author_nick    0
created_at     0
replies        0
combined       0
vector         0
dtype: int64
데이터 프레임 전체에서 누락된 값의 총 개수: 0


In [77]:
#Doc2Vec 학습하기
model = Doc2Vec(vector_size=128, window=2, min_count=1, workers=4, epochs=300, sample=1e-5 , seed=42, alpha=0.01, min_alpha=0.0001)
 
model.build_vocab(tagged_data)  #단어사전 학습 

print("vocab 사이즈 : ", len(model.wv.index_to_key))  #단어사전의 크기 

vocab 사이즈 :  50623


In [78]:
#단어 빈도수 확인
print("Word Frequencies:")
for word in model.wv.index_to_key:
    count = model.wv.get_vecattr(word, "count")
    print(f"Word {word}: {count}")

Word Frequencies:
Word 아이: 62541
Word 어요: 50677
Word 학원: 43460
Word 수학: 22107
Word 수업: 20977
Word 선생님: 17810
Word 어서: 17780
Word 내신: 17594
Word 생각: 16652
Word 영어: 16101
Word 공부: 15772
Word 정도: 15629
Word 부탁: 15343
Word 까지: 15325
Word 학교: 15272
Word 시간: 15020
Word 학년: 14484
Word 추천: 13898
Word 문제: 12184
Word 지금: 12161
Word 부터: 11496
Word 혹시: 11373
Word 고민: 11284
Word 엄마: 11103
Word 면서: 10989
Word 다른: 10891
Word 시험: 10659
Word 려고: 10320
Word 수능: 10008
Word 이번: 9860
Word 저희: 9585
Word 국어: 9584
Word 드립: 9584
Word 보고: 9306
Word 하다: 9186
Word 이나: 9011
Word 오늘: 8931
Word 고등: 8796
Word 아들: 8751
Word 해주: 8386
Word 서요: 8342
Word 하나: 8331
Word 그냥: 8294
Word 남편: 8218
Word 중등: 8124
Word 정말: 8121
Word 다가: 8055
Word 준비: 8037
Word 어디: 7908
Word 시작: 7845
Word 가요: 7780
Word 학생: 7771
Word 고요: 7406
Word 보다: 6926
Word 고3: 6893
Word 감사: 6814
Word 이제: 6790
Word 걱정: 6764
Word 선택: 6741
Word 더니: 6590
Word 고1: 6580
Word 한번: 6570
Word 요즘: 6492
Word 아보: 6409
Word 중3: 6393
Word 과학: 6340
Word 선행: 6278
Word 과목: 6242


In [79]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [80]:
# get_vector 함수는 기존과 동일
def get_vector(model, item_idx):
    try:
        return model.docvecs[item_idx] #서 식별자(item_idx)에 해당하는 벡터를 모델에서 추출하는 역할
    except KeyError:
        return None  # 해당 키가 없을 경우 None 반환
    


# 모델이 순수 숫자 문자열 태그를 사용하므로, 'item_idx'만을 사용하여 벡터 검색
df['vector'] = df.apply(lambda row: get_vector(model, str(row['item_idx'])) if row['combined'] else None, axis=1)

# vector_df 생성 시에도 동일한 방식 적용
vector_df = pd.DataFrame({
    'item_idx': df['item_idx'],
    # 여기서도 'item_idx'만을 사용하여 벡터 검색
    'vector': df.apply(lambda row: get_vector(model, str(row['item_idx'])) if row['combined'] else None, axis=1)
})

print(vector_df)

#combined를 벱터화한것 

/var/folders/xz/5z38xhnx3798ywm68t1fzzd40000gn/T/ipykernel_33272/283797180.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  return model.docvecs[item_idx] #서 식별자(item_idx)에 해당하는 벡터를 모델에서 추출하는 역할


        item_idx                                             vector
0           4561  [0.6086013, 0.47499812, -0.26751295, -0.116748...
1           5684  [0.39927167, -0.34114057, -0.73800105, 0.56053...
2            191  [-0.0073037115, -0.2437775, 0.25304675, 0.7326...
3           7408  [0.3709748, 0.50565994, -1.1514426, 0.04925793...
4           8194  [-0.5066662, 0.007347686, 0.16918911, 1.210750...
...          ...                                                ...
122440    545008  [1.1912905, 0.058996793, -0.206127, 0.43094155...
122441    544947  [-0.21351588, -0.18485914, -0.37671673, -0.072...
122442    544919  [0.27543083, 0.64377177, -0.079306595, 0.53197...
122444    545102  [-0.18672772, -0.20815891, -0.27428073, -0.264...
122445    545095  [0.021963049, 0.57581437, -0.6052079, 0.650434...

[82119 rows x 2 columns]


In [81]:
num_of_vectors = vector_df['vector'].notna().sum()
print(f"The total number of items with vectors: {num_of_vectors}")

The total number of items with vectors: 82119


In [89]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [90]:
 #유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np   #numpy : 파이썬 수치 연산 라이브러리

In [92]:
# 데이터 프레임에서 item_idx가 query_item_idx와 일치하는 행을 찾고 해당 행의 벡터 열 값 반환 
def get_query_vector(df, query_item_idx):
    # 일치하는 행 찾기 
    query_vector = df[df['item_idx'] == query_item_idx]['vector'].values
    # 해당 행의 'vector'열 값 가져오기
    return query_vector[0] if len(query_vector) > 0 else None

# 함수
def recommend_similar_items(query_item_idx, df, top_n=5):
    # 쿼리 아이템의 벡터 가져오기
    query_vector = get_query_vector(df, query_item_idx)
    if query_vector is None:
        return None  # 쿼리 아이템의 벡터를 찾을 수 없는 경우

    # 콘텐츠가 있는 아이템만 선택하여 벡터 추출하고 차원을 확인
    valid_items = df[df['contents'].notnull()]
    valid_vectors = valid_items['vector'].tolist()

    # 유효한 벡터만 선택하여 128차원 벡터만 추출
    valid_vectors_128 = [v for v in valid_vectors if v is not None and len(v) == 128]

    # 리스트 형태의 벡터를 NumPy 배열로 변환 (변환해야 계산 가능)
    item_vectors = np.array(valid_vectors_128)

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity([query_vector], item_vectors)
    similarity_scores = similarity_scores[0]  # 2D 배열을 1D로 변환

    # 유사도 기준으로 상위 N개 아이템 인덱스 추출 (쿼리 아이템 제외)
    similar_items_idx = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    
    # 추천 아이템 정보 출력
    recommended_items = valid_items.iloc[similar_items_idx][['item_idx', 'vector', 'contents']]
    recommended_items['similarity'] = similarity_scores[similar_items_idx]

    return recommended_items


In [93]:
# 사용 예시
query_item_idx = 7408
   # 예시
recommendations = recommend_similar_items(query_item_idx, df)
print(recommendations)

       item_idx                                             vector  \
3          7408  [0.3709748, 0.50565994, -1.1514426, 0.04925793...   
89702    495708  [-0.041794106, -0.03668289, -0.36488253, 0.068...   
76222    463425  [0.054472964, 0.04377276, -0.04709559, -0.2215...   
65262    417444  [0.30552524, 0.55082816, -0.42337686, 0.386309...   
498        7796  [0.70549226, 0.9539472, -0.07447567, -0.596695...   

                                                contents  similarity  
3      예비중1 이안어학원에 대한 들리는 평판이 좋기도 했고, 또 마침 아이시간표 조절할일...    1.000000  
89702  아이가 다니는 PEAI 학원에서 겨울방학 특강처럼 CRD 인문고전 특강모용상 원장님...    0.457796  
76222  ILE중등 주 2회반은 1회 수업 시간이 어떻게되나요? 3시간 정도 되는지요? 1회...    0.440393  
65262  예비 중1 입니다 청담 이글 다니다가 도저히 writing 못하겠다해서 그만두고,최...    0.439642  
498    ILE 다닌지 1년된 예비 초6 엄마입니다.일반적으로, 예비초4에 청담 par라면 ...    0.434962  


In [94]:
#엘라스틱서치 접속 설치
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [96]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch import helpers

In [103]:
# df는 데이터프레임, 'item_idx'와 'd2v_vector' 필드
data_for_es = [
    {   "_op_type": "update",  # update 하면 새롭게 안하고 업데이트 
        "_index": "items1", #문서가 속할 인덱스
        "_id": str(row['item_idx']),
        "doc": {
            "d2v_vector": [float(value) for value in row['vector']] if row['vector'] is not None else None
            # 'd2v_vector' 필드에 대한 값을 리스트로 제공
        }
    }
    for idx, row in df.iterrows()
    if row['vector'] is not None
]

In [104]:
# Elasticsearch 연결
es = Elasticsearch(hosts=[{'host': 'my-deployment-5d8a0f.es.us-east-1.aws.found.io', 'port': 9243,'scheme': 'https'}],
                  basic_auth=('elasticuser', 'hy%^&2022',),request_timeout=120)
#timeout: 서버에 요청이 완료될 때까지 더 많으 시간 부여

In [105]:
# bulk를 사용하여 데이터를 한 번에 업로드
#데이터 색인화
helpers.bulk(es, data_for_es,chunk_size=500)

(82119, [])